# NP2 channel-map geometry checker (`channel_positions.npy`)

This notebook helps you verify that each probe's Kilosort geometry matches the channel map you intended to use.

It will:
- load `channel_positions.npy` (and optionally `channel_map.npy`),
- plot the physical channel layout,
- summarize x/y spacing statistics,
- estimate a **suggested** `nChannelsIsoDist` range for BombCell.


In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 120


## 1) Configure your Kilosort folders

Set one folder per probe (A/C/D etc.). Each folder should contain `channel_positions.npy`.


In [ ]:
# Edit these paths
probe_dirs = {
    "probe_A": Path("/path/to/probe_A/ks4"),
    "probe_C": Path("/path/to/probe_C/ks4"),
    "probe_D": Path("/path/to/probe_D/ks4"),
}


## 2) Helpers

In [ ]:
def load_geometry(ks_dir: Path):
    ks_dir = Path(ks_dir)
    pos = np.load(ks_dir / "channel_positions.npy")

    cmap_file = ks_dir / "channel_map.npy"
    if cmap_file.exists():
        cmap = np.load(cmap_file).squeeze()
    else:
        cmap = None

    if pos.ndim != 2 or pos.shape[1] != 2:
        raise ValueError(f"Expected (n_channels,2) channel_positions in {ks_dir}, got {pos.shape}")

    return pos, cmap


def nearest_neighbor_distances(channel_positions):
    cp = np.asarray(channel_positions, dtype=float)
    d = cp[:, None, :] - cp[None, :, :]
    dist = np.sqrt((d**2).sum(axis=2))
    np.fill_diagonal(dist, np.inf)
    return dist.min(axis=1)


def summarize_geometry(channel_positions):
    cp = np.asarray(channel_positions, dtype=float)
    ux = np.unique(np.round(cp[:, 0], 6))
    uy = np.unique(np.round(cp[:, 1], 6))

    dx = np.diff(np.sort(ux)) if ux.size > 1 else np.array([])
    dy = np.diff(np.sort(uy)) if uy.size > 1 else np.array([])
    nn = nearest_neighbor_distances(cp)

    summary = {
        "n_channels": cp.shape[0],
        "n_unique_x": int(ux.size),
        "n_unique_y": int(uy.size),
        "median_dx": float(np.median(dx)) if dx.size else np.nan,
        "median_dy": float(np.median(dy)) if dy.size else np.nan,
        "median_nn_dist": float(np.median(nn)),
        "p90_nn_dist": float(np.percentile(nn, 90)),
    }
    return summary, nn


def suggest_nChannelsIsoDist(summary):
    """
    Heuristic suggestion:
    - sparse maps (large nearest-neighbor spacing) -> larger neighborhood
    - dense maps -> default is often fine
    """
    p90 = summary["p90_nn_dist"]
    if np.isnan(p90):
        return "unknown", "Could not estimate spacing"
    if p90 >= 40:
        return "8-12", "Sparse/interleaved geometry: use more nearby channels"
    if p90 >= 25:
        return "6-8", "Moderately sparse geometry: consider increasing from default 4"
    return "4-6", "Dense geometry: default 4 is usually reasonable"


def plot_geometry(channel_positions, title=""):
    cp = np.asarray(channel_positions)
    fig, ax = plt.subplots(figsize=(5, 9))
    ax.scatter(cp[:, 0], cp[:, 1], s=14)
    ax.set_xlabel("x (um)")
    ax.set_ylabel("y (um)")
    ax.set_title(title)
    ax.grid(alpha=0.2)
    ax.set_aspect('equal')
    plt.show()


## 3) Run geometry checks for all probes

In [ ]:
results = {}

for name, ks_dir in probe_dirs.items():
    print(f"\n=== {name} ===")
    if not (ks_dir / "channel_positions.npy").exists():
        print(f"Missing channel_positions.npy: {ks_dir}")
        continue

    channel_positions, channel_map = load_geometry(ks_dir)
    summary, nn = summarize_geometry(channel_positions)
    rec, note = suggest_nChannelsIsoDist(summary)

    results[name] = {
        "ks_dir": ks_dir,
        "summary": summary,
        "nChannelsIsoDist_recommendation": rec,
        "recommendation_note": note,
    }

    print(f"Kilosort folder: {ks_dir}")
    print(f"channels: {summary['n_channels']}")
    print(f"unique x: {summary['n_unique_x']}, unique y: {summary['n_unique_y']}")
    print(f"median dx: {summary['median_dx']:.2f} um")
    print(f"median dy: {summary['median_dy']:.2f} um")
    print(f"median NN distance: {summary['median_nn_dist']:.2f} um")
    print(f"p90 NN distance: {summary['p90_nn_dist']:.2f} um")
    print(f"Suggested nChannelsIsoDist: {rec} ({note})")

    plot_geometry(channel_positions, title=f"{name}: channel_positions")


## 4) Optional: quick comparison table

In [ ]:
try:
    import pandas as pd
    rows = []
    for name, info in results.items():
        row = {"probe": name, **info["summary"],
               "nChannelsIsoDist_recommendation": info["nChannelsIsoDist_recommendation"],
               "note": info["recommendation_note"]}
        rows.append(row)
    display(pd.DataFrame(rows).sort_values("probe"))
except Exception as e:
    print("Pandas table skipped:", e)


## How to interpret for your specific NP2 patterns

Given your description:
- **Probe A** (`ON,OFF,OFF,OFF` interleaving): likely sparsest effective local neighborhood → start around **8** (test 8–12).
- **Probe C** (`ON,OFF` interleaving): moderate sparsity → start around **6** (test 6–8).
- **Probe D** (contiguous ON region): densest local neighborhood → default **4** is usually acceptable (test 4–6).

Use the printed spacing stats and a small-unit QC comparison to decide final values.
